In [1]:
import numpy as np
import pandas as pd
import random

from functions import *

# 1. Hashing

## Sequentially adding data to the HLL

In [2]:
log2m, bits = 6, 32

HLL_6 = HyperLogLog(log2m=log2m, bits=bits)

hll_6 = HLL_6.structure('hash.txt')

print(hll_6)

[24, 22, 23, 21, 21, 23, 24, 26, 23, 23, 22, 23, 20, 26, 23, 21, 22, 22, 21, 21, 24, 23, 20, 21, 23, 21, 24, 20, 26, 21, 22, 23, 21, 19, 24, 22, 24, 23, 23, 22, 22, 22, 24, 22, 25, 24, 23, 20, 22, 23, 22, 23, 20, 22, 20, 22, 21, 19, 20, 22, 26, 21, 25, 23]


## Cardinality and error of the filter

In [6]:
estimate_6 = HLL_6.cardinality(hll_6)
error_6 = HLL_6.error()

print('Total bits: {}\nBits for the buckets: {}'.format(bits, log2m))
print('Estimate: {}\nError: {}'.format(estimate_6, error_6))

Total bits: 32
Bits for the buckets: 11
Estimate: 133851947
Error: 0.13


### Observation
Maybe we can do a little bit better by increasing the lenght of the root to 11 bits. This seems to be a good point for the tradeoff between error and efficiency!

In [5]:
log2m, bits = 11, 32

HLL_11 = HyperLogLog(log2m=log2m, bits=bits)

hll_11 = HLL_11.structure('hash.txt')

estimate_11 = HLL_11.cardinality(hll_11)

error_11 = HLL_11.error()

print('Total bits: {}\nBits for the buckets: {}'.format(bits, log2m))
print('Estimate: {}\nError: {:.3f}'.format(estimate_11, error_11))

Total bits: 32
Bits for the buckets: 11
Estimate: 123568357
Error: 0.023
